# Equations of Motion for a Rigid Body


In [2]:
:dep ndarray = { version = "0.16.1" }
:dep plotly = { version = "0.10.0" }
:dep nalgebra = { version = "0.33.1" }
extern crate ndarray;
extern crate plotly;
extern crate nalgebra;

use ndarray::{Array, Array2};
use nalgebra::{SMatrix, SVector};
use plotly::common::Mode;
use plotly::layout::{Layout};
use plotly::{Plot, Scatter};

In [61]:
// Define the state vector
// 0 u
// 1 v
// 2 w
// 3 p
// 4 q
// 5 r

let mut x = SVector::<f64, 6>::zeros();
let mut x_dot = SVector::<f64, 6>::zeros();

// x[4] = 20.0 * std::f64::consts::PI / 180.0;
x[5] = 10.0 * std::f64::consts::PI / 180.0;

In [75]:
let mass = 10.0;  // kg
let Ixx = 0.0167; // kg m^2
let Iyy = 0.0833; // kg m^2
let Izz = 0.125;  // kg m^2
let Ixy = 0.0;
let Ixz = 0.0;
let Iyz = 0.0;

let I = SMatrix::<f64, 3, 3>::new(
    Ixx, -Ixy, -Ixz,
    -Ixy, Iyy, -Iyz,
    -Ixz, -Iyz, Izz
);

let I_inv = I.try_inverse().unwrap();




In [74]:
let Fx = 10.0; // N
let Fy = 0.0; // N
let Fz = 0.0; // N

let Mx = 0.0;
let My = 0.0;
let Mz = 0.0;

let F = SVector::<f64, 3>::new(Fx, Fy, Fz);
let M = SVector::<f64, 3>::new(Mx, My, Mz);

// I'll try to do the coriolis stuff here
let omega = SVector::<f64, 3>::new(x[3], x[4], x[5]);
let part1 = I * omega;
let coriolis = omega.cross(&part1);

let omega_dot = I_inv * (M - coriolis);

x_dot[0] = (Fx / mass) + x[2] * x[4] - x[1] * x[5];
x_dot[1] = (Fy / mass) + x[0] * x[5] - x[2] * x[3];
x_dot[2] = (Fz / mass) + x[1] * x[3] - x[0] * x[4];
x_dot[3] = omega_dot[0];
x_dot[4] = omega_dot[1];
x_dot[5] = omega_dot[2];

In [73]:
println!("{}", x_dot);



  ┌   ┐
  │ 1 │
  │ 0 │
  │ 0 │
  │ 0 │
  │ 0 │
  │ 0 │
  └   ┘




In [131]:
#[derive(Debug)]
struct RigidBody {
    mass: f64,
    I: SMatrix::<f64, 3, 3>,
    I_inv: SMatrix::<f64, 3, 3>,
    x: SVector::<f64, 6>,
}

impl RigidBody {
    fn new(I: SMatrix::<f64, 3, 3>, mass: f64, initial_state: SVector::<f64, 6>) -> Self {

        let I_inv = I.try_inverse().unwrap();
        
        Self {
            mass,
            I,
            I_inv,
            x: initial_state,
        }
    }

    fn calculate_state_derivative(&self, x: &SVector::<f64, 6>, F: &SVector::<f64, 3>, M: &SVector::<f64, 3>) -> SVector::<f64, 6> {
        // I'll try to do the coriolis stuff here
        let omega = SVector::<f64, 3>::new(x[3], x[4], x[5]);
        let part1 = self.I * omega;
        let coriolis = omega.cross(&part1);
        
        let omega_dot = self.I_inv * (M - coriolis);

        let mut x_dot = SVector::<f64, 6>::zeros();
        x_dot[0] = (F[0] / self.mass) + x[2] * x[4] - x[1] * x[5];
        x_dot[1] = (F[1] / self.mass) + x[0] * x[5] - x[2] * x[3];
        x_dot[2] = (F[2] / self.mass) + x[1] * x[3] - x[0] * x[4];
        x_dot[3] = omega_dot[0];
        x_dot[4] = omega_dot[1];
        x_dot[5] = omega_dot[2];
        x_dot
    }

    fn propagate_state(&self, dt: f64, F: &SVector::<f64, 3>, M: &SVector::<f64, 3>) -> SVector::<f64, 6> {
        // Runge Kutta

        let x = self.x.clone();
        let k1 = self.calculate_state_derivative(&x, F, M);
        
        let x1 = x + (0.5 * dt) * k1;
        let k2 = self.calculate_state_derivative(&x1, F, M);
        
        let x2 = x + (0.5 * dt) * k2;
        let k3 = self.calculate_state_derivative(&x2, F, M);
        
        let x3 = x + dt * k3;
        let k4 = self.calculate_state_derivative(&x3, F, M);
        
        let x4 = x + (dt / 6.0) * (k1 + 2.0 * k2 + 2.0 * k3 + k4);

        x4
        
    }

    fn run(&mut self, steps: u64, dt: f64) {
        let Fx = 0.0; // N
        let Fy = 0.0; // N
        let Fz = 0.0; // N
        
        let Mx = 0.0;
        let My = 0.0;
        let Mz = 0.0;
        
        let F = SVector::<f64, 3>::new(Fx, Fy, Fz);
        let M = SVector::<f64, 3>::new(Mx, My, Mz);

        for i in 0..steps {
            self.x = self.propagate_state(dt, &F, &M);
        }
    }
}

In [134]:
let mut initial_state = SVector::<f64, 6>::zeros();
initial_state[0] = 1.0;
initial_state[5] = 90.0 * std::f64::consts::PI / 180.0;
let mut body = RigidBody::new(I, mass, initial_state);
body.run(100, 0.01);
body.x

[[7.968559527504748e-10, 0.9999999999895686, 0.0, 0.0, 0.0, 1.5707963267948966]]